<a href="https://colab.research.google.com/github/lsteffenel/M2Atmo_et_Climat/blob/main/14_KerasTCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime

import pandas as pd


def get_df_until_2021_02_01():
    df = pd.read_csv(
        'https://github.com/lsteffenel/M2Atmo_et_Climat/raw/refs/heads/main/data/weatherAUS_until_2016_01_01.csv',
        date_parser = lambda d: datetime.strptime(d, '%Y-%m-%d'),
        index_col = 'Date'
    )
    return df


In [ ]:
df=get_df_until_2021_02_01()
df=df.drop(['RainToday'],axis=1)
df=df[df['Location']=='Sydney']
df=df.drop(['Location'],axis=1)
df=df[['MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','Humidity9am','Humidity3pm','Cloud9am','Cloud3pm','Temp9am','Temp3pm']]

<ipython-input-1-9dbb0f80784a>:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


In [ ]:
df.iloc[5:10]


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
Date,,,,,,,,,,,
2008-02-06,20.2,27.2,1.6,2.6,8.6,69.0,62.0,6.0,6.0,23.8,26.0
2008-02-07,18.6,26.3,6.2,5.2,5.2,75.0,80.0,4.0,7.0,21.7,22.3
2008-02-08,17.2,22.3,27.6,5.8,2.1,77.0,61.0,7.0,8.0,18.9,21.1
2008-02-09,16.4,20.8,12.6,4.8,3.0,92.0,91.0,7.0,7.0,17.1,16.5
2008-02-10,14.6,24.2,8.8,4.4,10.1,80.0,53.0,4.0,2.0,17.2,23.3


In [ ]:
!pip uninstall -y keras-tcn

Found existing installation: keras-tcn 3.5.0
Uninstalling keras-tcn-3.5.0:
  Successfully uninstalled keras-tcn-3.5.0


In [ ]:
!git clone https://github.com/philipperemy/keras-tcn.git

Cloning into 'keras-tcn'...
remote: Enumerating objects: 1995, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1995 (delta 91), reused 143 (delta 80), pack-reused 1821 (from 1)
Receiving objects: 100% (1995/1995), 5.87 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (1049/1049), done.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tcn import TCN
import numpy as np

# Example function for preparing data
def prepare_data(data, target_col, sequence_length):
    data = data.dropna()
    y = []
    y = data[target_col]
    data = data.drop([target_col],axis=1)
    X = []
    for i in range(len(data) - sequence_length):
        X.append(data.iloc[i:i + sequence_length-1])  # All columns except target
    return np.array(X), np.array(y)

# Hypothetical data (replace this with your actual dataset)
data = df

# Target column index (replace with your actual rainfall column index)
target_col = 'Rainfall'
sequence_length = 12

X, y = prepare_data(data, target_col, sequence_length)


In [ ]:
print(X[1])

[[19.5 25.6  3.4  2.7 83.  73.   7.   7.  22.4 24.8]
 [21.6 24.5  2.4  0.1 88.  86.   7.   8.  23.5 23. ]
 [20.2 22.8  2.2  0.  83.  90.   8.   8.  21.4 20.9]
 [20.2 27.2  2.6  8.6 69.  62.   6.   6.  23.8 26. ]
 [18.6 26.3  5.2  5.2 75.  80.   4.   7.  21.7 22.3]
 [17.2 22.3  5.8  2.1 77.  61.   7.   8.  18.9 21.1]
 [16.4 20.8  4.8  3.  92.  91.   7.   7.  17.1 16.5]
 [14.6 24.2  4.4 10.1 80.  53.   4.   2.  17.2 23.3]
 [16.4 23.9  6.4  8.  78.  53.   7.   8.  18.9 23.7]
 [18.9 27.3  6.8  6.7 68.  67.   7.   7.  22.9 24.7]
 [18.4 22.8  7.   3.3 87.  70.   8.   7.  20.9 21. ]]


In [ ]:
print(sequence_length, X.shape[2])

12 10


In [ ]:
print(y[1])

6.0


In [ ]:

# Define TCN model

model = Sequential([
    TCN(input_shape=(sequence_length, X.shape[2]), nb_filters=64, kernel_size=3, dilations=[1, 2, 4], dropout_rate=0.2),
    Dense(1, activation='linear')  # Couche de sortie
])

#model = Sequential([
#    #TCN(nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], dropout_rate=0.2, input_shape=(sequence_length, 1)),  # TCN layer
#    TCN(input_shape=(sequence_length, X.shape[2])),
#    Dense(1)  # Regression output
#])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Predict example
predicted_rainfall = model.predict(X[:1])
print("Predicted rainfall:", predicted_rainfall)

AttributeError: 'tuple' object has no attribute 'as_list'

In [ ]:
# Example function for preparing data
def prepare_data2(data, target_col, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :-1])  # All columns except target
        y.append(data[i + sequence_length, target_col])  # Target column
    return np.array(X), np.array(y)

# Hypothetical data (replace this with your actual dataset)
import numpy as np
data = np.random.rand(1000, 12)  # 1000 rows, 12 features

In [ ]:
target_col = 3
sequence_length = 10

X2, y2 = prepare_data2(data, target_col, sequence_length)

In [ ]:
print(X2)

[[[0.58758676 0.9916094  0.4960126  ... 0.34443938 0.29869108 0.48569474]
  [0.46187446 0.3160829  0.56868153 ... 0.32885654 0.89766362 0.9979239 ]
  [0.80564082 0.12563287 0.55789744 ... 0.94418636 0.58774443 0.09338227]
  ...
  [0.35979796 0.86961301 0.67997456 ... 0.50021758 0.12335131 0.27503134]
  [0.05299114 0.60157819 0.50462221 ... 0.60619698 0.00241427 0.36982567]
  [0.60303447 0.97675621 0.21753028 ... 0.6785157  0.82967613 0.04963279]]

 [[0.46187446 0.3160829  0.56868153 ... 0.32885654 0.89766362 0.9979239 ]
  [0.80564082 0.12563287 0.55789744 ... 0.94418636 0.58774443 0.09338227]
  [0.77452962 0.27740781 0.0729767  ... 0.88622544 0.09188532 0.44263084]
  ...
  [0.05299114 0.60157819 0.50462221 ... 0.60619698 0.00241427 0.36982567]
  [0.60303447 0.97675621 0.21753028 ... 0.6785157  0.82967613 0.04963279]
  [0.00503373 0.28097537 0.15891852 ... 0.08514505 0.62118867 0.50541978]]

 [[0.80564082 0.12563287 0.55789744 ... 0.94418636 0.58774443 0.09338227]
  [0.77452962 0.277407

In [ ]:
def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, 10, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0  # we introduce the target in the first timestep of the sequence.
    y_train[pos_indices, 0] = 1.0  # the task is to see if the TCN can go back in time to find it.
    return x_train, y_train

x3, y3 = get_x_y()
print(x3.shape)

(1000, 10, 1)
